# Funciones de Utilidad

Se busca reutilización y dar legibilidad al código

In [ ]:
import numpy as np  # pa arrays
import time         # medición tiempo
import os

np.random.seed(42) # semilla para números aleatorios

### Generación de Datos Sintéticos Ordenados

Para probar los algoritmos de índice (RMI y B-Tree), necesitamos conjuntos de datos. Para los índices, es fundamental que los datos estén ordenados.

In [ ]:
# --- Función de Generación de enteros ordenados ---
def generate_sorted_data(size, min_val=0, max_val=None):
    """
    Genera un array de enteros ordenados aleatoriamente.

    Args:
        size (int): El número de elementos en el array.
        min_val (int): El valor mínimo posible para los enteros.
        max_val (int, optional): El valor máximo posible para los enteros.
                                 Si es None, se calculará un valor por defecto.

    Returns:
        np.array: Un array de enteros de numpy, ordenado de forma ascendente.
    """
    if max_val is None:
        # Si no se especifica max_val, lo hacemos el doble del tamaño para tener un buen rango.
        max_val = size * 2
    
    # Generamos números aleatorios y luego los ordenamos.
    # El orden es crucial para los índices como RMI y B-Tree.
    data = np.random.randint(min_val, max_val, size=size)
    data.sort() # np.sort() crea una copia, .sort() ordena in-place (más eficiente)
    return data

# --- Demostración de uso ---
print("Generando un pequeño dataset de ejemplo:")
small_data = generate_sorted_data(size=10)
print(small_data)

print("\nGenerando un dataset más grande para simulación (solo mostrando los primeros 10):")
large_data = generate_sorted_data(size=1_000_000)
print(large_data[:10])
print("...")
print(large_data[-10:])

Utilidad 2: Medición de Tiempo de Ejecución

Cuando comparamos algoritmos, queremos saber cuál es más rápido. Para eso, necesitamos medir el tiempo que tardan en ejecutarse ciertas operaciones (como la construcción del índice o una búsqueda).

Usaremos la librería time de Python para esto. La idea es registrar el tiempo antes de una operación y después de ella, y luego calcular la diferencia.

In [ ]:
# --- Función de Medición de Tiempo ---

def measure_time(func, *args, **kwargs):
    """
    Mide el tiempo de ejecución de una función dada.

    Args:
        func (callable): La función a medir.
        *args: Argumentos posicionales para la función.
        **kwargs: Argumentos de palabra clave para la función.

    Returns:
        tuple: Una tupla (result, elapsed_time) donde 'result' es el
               resultado de la función y 'elapsed_time' es el tiempo en segundos.
    """
    start_time = time.perf_counter() # Una medida de tiempo precisa
    result = func(*args, **kwargs)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    return result, elapsed_time

# --- Demostración de uso ---
print("\nDemostrando la medición de tiempo:")

# Una función de ejemplo que tarda un poco
def long_running_task(n):
    sum_val = 0
    for i in range(n):
        sum_val += i
    return sum_val

# Medimos el tiempo de nuestra tarea
result, time_taken = measure_time(long_running_task, 10_000_000)

print(f"El resultado de la tarea fue: {result}")
print(f"La tarea tardó: {time_taken:.6f} segundos.")

# Otra demostración, generando un dataset
_, gen_time = measure_time(generate_sorted_data, 5_000_000)
print(f"Generar 5 millones de datos ordenados tardó: {gen_time:.6f} segundos.")

Utilidad 3: Carga y Guardado de Datos (Opcional pero Útil)

Para conjuntos de datos muy grandes, generarlos cada vez que ejecutamos el código puede ser lento. Es más eficiente generarlos una vez y guardarlos en un archivo, para luego cargarlos rápidamente cuando los necesitemos.

Usaremos numpy para guardar y cargar arrays en un formato binario (.npy), que es muy eficiente

In [ ]:
# --- Funciones de Carga/Guardado de Datos ---

def save_data(data, filename="sample_dataset.npy", directory="data"):
    """
    Guarda un array de numpy en un archivo .npy.

    Args:
        data (np.array): El array a guardar.
        filename (str): El nombre del archivo.
        directory (str): El directorio donde guardar el archivo.
    """
    os.makedirs(directory, exist_ok=True) # Crea el directorio si no existe
    filepath = os.path.join(directory, filename)
    np.save(filepath, data)
    print(f"Datos guardados en: {filepath}")

def load_data(filename="sample_dataset.npy", directory="data"):
    """
    Carga un array de numpy desde un archivo .npy.

    Args:
        filename (str): El nombre del archivo.
        directory (str): El directorio del archivo.

    Returns:
        np.array: El array cargado. Retorna None si el archivo no existe.
    """
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        data = np.load(filepath)
        print(f"Datos cargados desde: {filepath}")
        return data
    else:
        print(f"Advertencia: Archivo {filepath} no encontrado.")
        return None

# --- Demostración de uso ---
print("\nDemostrando carga y guardado de datos:")

# Generar y guardar un dataset
test_data_to_save = generate_sorted_data(size=100)
save_data(test_data_to_save, filename="test_data.npy", directory="temp_data")

# Cargar el dataset que acabamos de guardar
loaded_test_data = load_data(filename="test_data.npy", directory="temp_data")

if loaded_test_data is not None:
    print(f"Primeros 5 elementos de los datos cargados: {loaded_test_data[:5]}")
    print(f"Verificación: ¿Los datos cargados son iguales a los guardados? {np.array_equal(test_data_to_save, loaded_test_data)}")
    
    # Limpieza: eliminar el archivo temporal
    os.remove(os.path.join("temp_data", "test_data.npy"))
    os.rmdir("temp_data") # Eliminar el directorio si está vacío
    print("Archivos temporales eliminados.")
else:
    print("No se pudo cargar el archivo de prueba.")

Conclusión: La Importancia de las Utilidades

Hemos visto cómo estas funciones auxiliares, aunque no son el "núcleo" de nuestros algoritmos de índice, son absolutamente fundamentales para el desarrollo, prueba y benchmarking de nuestro proyecto. Nos permiten:

    Preparar datos de manera consistente.
    Medir el rendimiento de forma precisa.
    Gestionar el almacenamiento de datos de manera eficiente.

Al organizar estas funcionalidades en un archivo utils.py dentro de nuestro módulo rmi/ (como lo indica la estructura del proyecto), nuestro código principal se mantiene limpio y centrado en la lógica del RMI, delegando estas tareas a las herramientas adecuadas